In [46]:
import pandas as pd
import random

In [49]:
stocks_info = pd.read_csv(r'data\sp500-with-gics.csv')
# stocks_info

In [51]:
unique_sectors_list = stocks_info['Sector'].unique().tolist()
unique_sub_industries_list = stocks_info['SubIndustry'].unique().tolist()
# unique_sectors_list


In [53]:
positive_quarterly_returns = pd.read_csv(r"data\positive_quarterly_return_rates.csv")
tickers = positive_quarterly_returns.columns.to_list()
tickers.remove("Date")

positive_stocks_info = stocks_info[stocks_info['Ticker'].isin(tickers)]

melted_df = pd.melt(positive_quarterly_returns, id_vars=['Date'], var_name='Ticker', value_name='Return')

# melted_df

In [54]:
ticker_and_sector_df = stocks_info[['Ticker', 'Sector']]
merged_df = melted_df.merge(ticker_and_sector_df, on='Ticker', how='left')
merged_df

,Date,Ticker,Return,Sector
0,2000-03-31 00:00:00+00:00,A,0.676857,Health Care
1,2000-06-30 00:00:00+00:00,A,0.361111,Health Care
2,2000-09-30 00:00:00+00:00,A,-0.248087,Health Care
3,2000-12-31 00:00:00+00:00,A,-0.313825,Health Care
4,2001-03-31 00:00:00+00:00,A,0.006180,Health Care
...,...,...,...,...
21307,2022-12-31 00:00:00+00:00,YUM,-0.060535,Consumer Discretionary
21308,2023-03-31 00:00:00+00:00,YUM,0.168929,Consumer Discretionary
21309,2023-06-30 00:00:00+00:00,YUM,0.054058,Consumer Discretionary
21310,2023-09-30 00:00:00+00:00,YUM,0.035287,Consumer Discretionary


In [43]:

sector_count = {}
sector_dict = {}
for sector in unique_sectors_list:
    sector_df = merged_df[merged_df["Sector"] == sector]
    positive_returns_pivot = sector_df.pivot(index='Date', columns='Ticker', values='Return').reset_index()

    stock_names_per_sector = positive_returns_pivot.columns.to_list()
    stock_names_per_sector.remove("Date")
    sector_count[sector] = len(stock_names_per_sector)
    sector_dict[sector] = stock_names_per_sector

    csv_filename = f"{sector}_positive_returns.csv"
    positive_returns_pivot.to_csv(r'data/positive_returns_per_sector/' + csv_filename, index=False)

In [55]:
# sector_count

In [56]:
# sector_dict

In [61]:
def get_n_random_stock_per_sector(stocks_num_per_sector, sector_dict):
    output_stocks_list = []
    for sector in sector_dict:
        max_stocks_per_sector = len(sector_dict[sector])
        if stocks_num_per_sector <= max_stocks_per_sector:
            output_stocks_list.append(random.sample(sector_dict[sector], stocks_num_per_sector))
        else:
            output_stocks_list.append(random.sample(sector_dict[sector], max_stocks_per_sector))
    return [stock for sector_stocks in output_stocks_list for stock in sector_stocks] # initially 'output_stocks_list' is a list of n-elements lists (one per sector)

In [77]:
stocks_num_per_sector = 2
n_stocks_per_sector = get_n_random_stock_per_sector(stocks_num_per_sector, sector_dict)
columns_to_keep = ["Date"] + n_stocks_per_sector
output_df = positive_quarterly_returns[columns_to_keep]
output_df.to_csv(r"data/random_data/n_stocks_per_sector.csv", index=False)


In [78]:
df = pd.read_csv(r'data\random_data\n_stocks_per_sector.csv')
df

,Date,CMI,JBHT,BDX,BIO,MSFT,SWKS,OMC,T,TJX,...,SPGI,SCHW,EMN,PPG,FRT,MAA,CPB,MNST,OKE,CVX
0,2000-03-31 00:00:00+00:00,0.195893,0.038648,-0.033333,-0.090452,0.250839,0.040372,0.088923,-0.098321,-0.149214,...,0.038889,-0.013333,0.208271,0.037838,0.063195,0.011236,-0.161744,0.000000,-0.089450,-0.095947
1,2000-06-30 00:00:00+00:00,-0.205801,0.083360,0.034273,0.182320,-0.220375,0.470647,-0.016579,-0.040978,0.174700,...,-0.226324,0.539384,0.046393,-0.109925,0.040391,0.031282,-0.170975,0.028984,0.021808,0.113223
2,2000-09-30 00:00:00+00:00,-0.232879,-0.012931,0.049626,-0.093458,-0.119669,-0.055480,-0.046827,0.048114,-0.218754,...,0.216852,-0.145171,0.036911,-0.126920,0.115118,0.100642,-0.044156,-0.014084,0.095322,-0.062673
3,2000-12-31 00:00:00+00:00,0.106366,-0.144105,-0.056677,-0.076289,-0.260938,-0.263610,-0.184021,0.083130,0.218365,...,0.198521,0.079308,-0.239365,-0.133279,-0.061714,0.000928,-0.115484,0.228578,0.477836,0.016826
4,2001-03-31 00:00:00+00:00,0.226378,0.265306,0.290754,0.433036,-0.266385,0.001946,0.070974,0.012824,0.217750,...,-0.123980,-0.255126,0.291668,0.162415,0.020951,-0.024719,0.371455,-0.290695,0.178212,-0.000944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2022-12-31 00:00:00+00:00,0.087161,0.015786,-0.080390,-0.148306,-0.070610,-0.029800,-0.001225,-0.243934,0.127718,...,-0.081048,0.148952,-0.155154,-0.026434,-0.055726,-0.089354,-0.008275,-0.058357,-0.041543,0.045039
92,2023-03-31 00:00:00+00:00,0.157324,0.067783,0.114175,-0.012645,-0.002014,0.036627,0.310116,0.199550,0.261326,...,0.067274,0.122347,0.112839,0.116725,0.139051,-0.011474,0.186840,0.137532,0.213016,0.155578
93,2023-06-30 00:00:00+00:00,-0.003701,0.006890,-0.032286,0.100348,0.201893,0.304340,0.162556,0.049626,0.001802,...,0.035810,-0.368679,0.038052,0.109317,-0.028031,-0.027554,-0.012180,0.057639,0.031685,-0.014585
94,2023-09-30 00:00:00+00:00,0.048530,0.054347,0.064460,-0.187729,0.179295,-0.042533,0.011249,-0.156853,0.072768,...,0.149177,0.125171,0.025823,0.069262,-0.002958,0.035888,-0.150950,0.062897,-0.026650,-0.065905
